In [1]:
from tensorflow import keras
import numpy as np
from matplotlib import pyplot as plt
import cv2
from pathlib import Path

In [2]:
paths = ["data/normal", "data/codiv19"]  
imgs = []  
labels = [] 

for idx, path in enumerate(paths):
    path_images = Path(path).glob("**/*.jpg")
    path_images = [str(x) for x in path_images]
    
    for path_image in path_images:
        img = cv2.imread(path_image, 0)  # Leer la imagen en modo de escala de grises
        img = img / 255  # Normalizar la imagen
        imgs.append(cv2.resize(img, (150, 150), cv2.INTER_AREA))  # Redimensionar y añadir a la lista
    
    labels.extend([idx] * len(path_images))  # Añadir etiquetas correspondientes

# Convertir las listas a arreglos de NumPy
imgs = np.array(imgs).reshape(-1, 150, 150, 1)  # Añadir dimensión de canal para imágenes en escala de grises
labels = np.array(labels)

print(f"Images shape: {imgs.shape}")
print(f"Num. of labels: {len(labels)}")

Images shape: (30, 150, 150, 1)
Num. of labels: 30


In [3]:
model = keras.Sequential([
    keras.layers.Input(shape=(150, 150, 1)),  
    keras.layers.Conv2D(32, (3, 3), padding="same", activation="relu"), 
    keras.layers.MaxPooling2D(pool_size=(2, 2)),  

    keras.layers.Conv2D(64, (3, 3), padding="same", activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),

    keras.layers.Conv2D(128, (3, 3), padding="same", activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),

    keras.layers.Flatten(),  
    
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.2),  

    keras.layers.Dense(2, activation="softmax") 
])

# Imprimir el resumen del modelo
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 150, 150, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 75, 75, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 75, 75, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 37, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 37, 37, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 18, 18, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 41472)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     5,308,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,401,474 (20.60 MB)

 Trainable params: 5,401,474 (20.60 MB)

 Non-trainable params: 0 (0.00 B)

None


In [4]:
from tensorflow.keras.utils import to_categorical

# Convertir etiquetas a formato one-hot encoding
#labels = to_categorical(labels, num_classes=2)

print(f"Shape de imágenes: {imgs.shape}")
print(f"Shape de etiquetas: {labels.shape}")

Shape de imágenes: (30, 150, 150, 1)
Shape de etiquetas: (30,)


In [5]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(
    imgs, labels,
    epochs=20,  
    batch_size=32,
    verbose=1
)

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.4333 - loss: 0.6984
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step - accuracy: 1.0000 - loss: 1.0396e-04
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step - accuracy: 1.0000 - loss: 3.9736e-08
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [12]:

# Función para cargar y preprocesar una imagen
def load_and_preprocess_image(image_path):
    # Cargar la imagen
    img = cv2.imread(image_path, 0)  # 0 para leer en escala de grises

    # Convertir la imagen a escala de grises si tiene 3 canales (RGB)
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


    # Redimensionar la imagen a (150, 150)
    img = cv2.resize(img, (150, 150))
    # Normalizar la imagen
    img = img / 255.0
    # Añadir una dimensión para los canales (la imagen debe tener forma (150, 150, 1))
    img = np.expand_dims(img, axis=-1)
    # Añadir una dimensión para que la forma sea (1, 150, 150, 1)
    img = np.expand_dims(img, axis=0)
    return img

# Función para hacer predicciones
def predict_image(model, image_path):
    # Cargar y preprocesar la imagen
    img = load_and_preprocess_image(image_path)
    # Realizar la predicción
    predictions = model.predict(img)
    # Obtener la clase con mayor probabilidad
    predicted_class = np.argmax(predictions)

    # Imprimir el resultado
    if predicted_class == 0:
        print("La imagen es de una persona normal (sin COVID-19).")
    else:
        print("La imagen es de una persona con COVID-19.")




In [16]:
# Ejemplo de uso:
image_path = 'data/covid19/C25.jpg'
predict_image(model, image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
La imagen es de una persona normal (sin COVID-19).
